In [1]:
import os
import numpy as np
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms


class CoronarySmallDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = os.listdir(image_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        SIZE = 256
        img_path = os.path.join(self.image_dir, self.images[idx])
        mask_path = os.path.join(self.mask_dir, self.images[idx])
        

        image = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
        image = cv2.cvtColor(image, cv2.COLOR_RGBA2RGB)
        image = cv2.resize(image, (SIZE, SIZE), cv2.INTER_NEAREST)
        
        mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)
        # mask = cv2.cvtColor(mask, cv2.COLOR_RGBA2RGB)
        mask = cv2.resize(mask, (SIZE, SIZE), cv2.INTER_NEAREST)
        
        if self.transform:
            image = self.transform(image)
            # mask = self.transform(mask)
            mask = torch.tensor(mask, dtype=torch.long)
        
        return image, mask


transform = transforms.Compose([
    transforms.ToTensor()
])

train_image_dir = '..\images\images_left\images_train\input_dicom'
train_mask_dir = '..\images\images_left\images_train\output'
val_image_dir = '..\images\images_left\images_val\input_dicom'
val_mask_dir = '..\images\images_left\images_val\output'
test_image_dir = '..\images\images_left\images_test\input_dicom'
test_mask_dir = '..\images\images_left\images_test\output'

train_dataset = CoronarySmallDataset(train_image_dir, train_mask_dir, transform=transform)
val_dataset = CoronarySmallDataset(val_image_dir, val_mask_dir, transform=transform)
test_dataset = CoronarySmallDataset(test_image_dir, test_mask_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)


In [4]:
from converter_RGB import convert_int_to_RGB
from large_RGB_model import UNet


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet()
model = model.to(device)

model.load_state_dict(torch.load('model_dicom_left_1.pth', map_location=device))
model.eval()

def show_image(type, image_name):
    dir = f"..\images\images_test\{type}"
    print(dir)
    if type == 'output':
        img = cv2.imread(os.path.join(dir, image_name), cv2.IMREAD_UNCHANGED)
        # img = img[:, :, 2]
        print("Oryginalnie: ", np.unique(img))
        
        print("Po resize:  ", np.unique(cv2.resize(img, (256, 256))))
        # print(img)
        img = convert_int_to_RGB(img)
        img = cv2.cvtColor(img, cv2.COLOR_BGRA2RGB)
    else:
        img = cv2.imread(os.path.join(dir, image_name), cv2.IMREAD_UNCHANGED)
        img = cv2.cvtColor(img, cv2.COLOR_RGBA2RGB)
    # img = cv2.resize(img, (256, 256))
    # np.set_printoptions(threshold=np.inf)
    cv2.imshow(type, img)

def predict(image_name):
    dir = '..\images\images_test\input_dicom'
    img = cv2.imread(os.path.join(dir, image_name), cv2.IMREAD_UNCHANGED)
    img = cv2.cvtColor(img, cv2.COLOR_RGBA2RGB)
    img = cv2.resize(img, (256, 256))
    img = transforms.ToTensor()(img).unsqueeze(0).to(device)

    with torch.no_grad():
        pred = model.predict(img)
        pred = pred.squeeze().cpu().numpy()
    print("Z sieci:    ", np.unique(pred))

    pred_image = convert_int_to_RGB(pred)
    pred_image = cv2.resize(pred_image, (512, 512))
    pred_image = cv2.cvtColor(pred_image, cv2.COLOR_BGR2RGB)
    cv2.imshow('pred', pred_image)
    cv2.waitKey(0)
   

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_30608\1758080196.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_dicom_left_1.pth',

In [20]:
# image_name = "131aedfhs6pnf1fvtvp49mhdb2fucqzo22_29.png"
image_name = "131aedfhs6pnf1fvtvp49mld7mqexnz322_36.png"
# image_name = "131aedfhs6pnf1fvtvp49mia892s56cf22_28.png"
# image_name = "131aedfhs6pnf1fvtvp49juwu7plj9dv22_40.png"
# image_name = "13c2ur549vohc0jat2dvu3xs7q1_18.png"
# image_name = "131aedfhs6pnf1fvtvp49mi8hml81goj22_37.png"

show_image("input_dicom", image_name)
show_image("output", image_name)
predict(image_name)

..\images\images_test\input_dicom
..\images\images_test\output
Oryginalnie:  [ 0  1  2  3  4 22]
Po resize:   [ 0  1  2  3  4  6  8 11 17 22]
Z sieci:     [ 0  1  2  3  4 22]


In [5]:
from sklearn.metrics import precision_score, recall_score, f1_score


def compute_accuracy(model, data_loader, device):
    model.eval()
    correct = 0
    correct_clear = 0
    total = 0
    total_clear = 0

    with torch.no_grad():
        for images, masks in data_loader:
            images = images.to(device)
            masks = masks.to(device)

            preds = model.predict(images)
            
            correct += (preds == masks).sum().item()
            correct_clear += (preds[masks != 0] == masks[masks != 0]).sum().item()
            total += masks.numel()
            total_clear += masks[masks != 0].numel()

    accuracy = correct / total
    accuracy_clear = correct_clear / total_clear
    return accuracy, accuracy_clear

In [6]:
# Do poprawy, teraz nie ma sensu
# Musiałoby dla każdej klasu osobno liczyć
def compute_precision_recall_f1(model, data_loader, device, num_classes=28):
    model.eval()
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for images, masks in data_loader:
            images = images.to(device)
            masks = masks.to(device)

            preds = model.predict(images)
            
            all_preds.append(preds.view(-1).cpu().numpy())
            all_targets.append(masks.view(-1).cpu().numpy())

    all_preds = np.concatenate(all_preds)
    all_targets = np.concatenate(all_targets)

    TP = np.zeros(num_classes)
    FP = np.zeros(num_classes)
    FN = np.zeros(num_classes)

    for i in range(num_classes):
        TP[i] = ((all_preds == i) & (all_targets == i)).sum()
        FP[i] = ((all_preds == i) & (all_targets != i)).sum()
        FN[i] = ((all_preds != i) & (all_targets == i)).sum()
    
    TP[0], FP[0], FN[0] = 0, 0, 0
    # print(TP, FP, FN)

    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    f1 = 2 * precision * recall / (precision + recall)
    # print(TP.sum(), FP.sum(), FN.sum())

    precision_weighted = TP.sum() / (TP.sum() + FP.sum())
    recall_weighted = TP.sum() / (TP.sum() + FN.sum())
    f1_weighted = 2 * precision_weighted * recall_weighted / (precision_weighted + recall_weighted)

    return precision, recall, f1, precision_weighted, recall_weighted, f1_weighted

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet().to(device)
model.load_state_dict(torch.load('model_dicom_left_1.pth', map_location=device))

train_accuracy = compute_accuracy(model, train_loader, device)
val_accuracy = compute_accuracy(model, val_loader, device)

print(f'Train Accuracy: {train_accuracy[0] * 100 :.2f}%')
print(f'Train Accuracy Clear: {train_accuracy[1] * 100 :.2f}%')
print()
print(f'Validation Accuracy: {val_accuracy[0] * 100 :.2f}%')
print(f'Validation Accuracy Clear: {val_accuracy[1] * 100 :.2f}%')

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_30608\2721329908.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_dicom_left_1.pth',

Train Accuracy: 96.04%
Train Accuracy Clear: 43.46%

Validation Accuracy: 94.74%
Validation Accuracy Clear: 28.88%


In [8]:
model.load_state_dict(torch.load('model_dicom_left_1.pth', map_location=device))
precision, recall, f1, precision_weighted, recall_weighted, f1_weighted = compute_precision_recall_f1(model, val_loader, device)

if np.isnan(precision).any():
    precision = np.nan_to_num(precision)
if np.isnan(recall).any():
    recall = np.nan_to_num(recall)
if np.isnan(f1).any():
    f1 = np.nan_to_num(f1)

for i in range(28):
    print(f'Class {i}: Precision: {precision[i] * 100:.2f}%, Recall: {recall[i] * 100:.2f}%, F1: {f1[i] * 100:.2f}%')

print(f'Average Precision: {np.mean(precision) * 100:.3f}%')
print(f'Average Recall: {np.mean(recall) * 100:.3f}%')
print(f'Average F1: {np.mean(f1) * 100:.3f}%')
print()
print(f'Weighted Precision: {precision_weighted * 100:.3f}%')
print(f'Weighted Recall: {recall_weighted * 100:.3f}%')
print(f'Weighted F1: {f1_weighted * 100:.3f}%')

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_30608\539058905.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_dicom_left_1.pth', 

Class 0: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 1: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 2: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 3: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 4: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 5: Precision: 67.79%, Recall: 47.24%, F1: 55.68%
Class 6: Precision: 53.98%, Recall: 52.00%, F1: 52.97%
Class 7: Precision: 52.16%, Recall: 44.49%, F1: 48.02%
Class 8: Precision: 57.35%, Recall: 33.29%, F1: 42.13%
Class 9: Precision: 33.99%, Recall: 14.62%, F1: 20.45%
Class 10: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 11: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 12: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 13: Precision: 50.43%, Recall: 49.48%, F1: 49.95%
Class 14: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 15: Precision: 25.94%, Recall: 41.16%, F1: 31.82%
Class 16: Precision: 100.00%, Recall: 0.01%, F1: 0.02%
Class 17: Precision: 52.67%, Recall: 43.64%, F1: 47.73%
Class 18: Precision: 22.19%, R

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_30608\3879195863.py:33: RuntimeWarning: invalid value encountered in divide
  precision = TP / (TP + FP)
C:\Users\eozoszy\AppData\Local\Temp\ipykernel_30608\3879195863.py:34: RuntimeWarning: invalid value encountered in divide
  recall = TP / (TP + FN)


In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet().to(device)
model.load_state_dict(torch.load('model_dicom_left_2.pth', map_location=device))

train_accuracy = compute_accuracy(model, train_loader, device)
val_accuracy = compute_accuracy(model, val_loader, device)

print(f'Train Accuracy: {train_accuracy[0] * 100 :.2f}%')
print(f'Train Accuracy Clear: {train_accuracy[1] * 100 :.2f}%')
print()
print(f'Validation Accuracy: {val_accuracy[0] * 100 :.2f}%')
print(f'Validation Accuracy Clear: {val_accuracy[1] * 100 :.2f}%')

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_30608\532320188.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_dicom_left_2.pth', 

Train Accuracy: 93.92%
Train Accuracy Clear: 10.93%

Validation Accuracy: 93.75%
Validation Accuracy Clear: 10.88%


In [10]:
model.load_state_dict(torch.load('model_dicom_left_2.pth', map_location=device))
precision, recall, f1, precision_weighted, recall_weighted, f1_weighted = compute_precision_recall_f1(model, val_loader, device)

if np.isnan(precision).any():
    precision = np.nan_to_num(precision)
if np.isnan(recall).any():
    recall = np.nan_to_num(recall)
if np.isnan(f1).any():
    f1 = np.nan_to_num(f1)

for i in range(28):
    print(f'Class {i}: Precision: {precision[i] * 100:.2f}%, Recall: {recall[i] * 100:.2f}%, F1: {f1[i] * 100:.2f}%')

print(f'Average Precision: {np.mean(precision) * 100:.3f}%')
print(f'Average Recall: {np.mean(recall) * 100:.3f}%')
print(f'Average F1: {np.mean(f1) * 100:.3f}%')
print()
print(f'Weighted Precision: {precision_weighted * 100:.3f}%')
print(f'Weighted Recall: {recall_weighted * 100:.3f}%')
print(f'Weighted F1: {f1_weighted * 100:.3f}%')

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_30608\346049334.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_dicom_left_2.pth', 

Class 0: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 1: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 2: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 3: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 4: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 5: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 6: Precision: 27.95%, Recall: 45.99%, F1: 34.77%
Class 7: Precision: 24.57%, Recall: 51.09%, F1: 33.19%
Class 8: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 9: Precision: 41.67%, Recall: 0.02%, F1: 0.03%
Class 10: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 11: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 12: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 13: Precision: 27.78%, Recall: 0.03%, F1: 0.05%
Class 14: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 15: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 16: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 17: Precision: 27.19%, Recall: 20.66%, F1: 23.48%
Class 18: Precision: 0.00%, Recall: 0.00%, F1

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_30608\3879195863.py:33: RuntimeWarning: invalid value encountered in divide
  precision = TP / (TP + FP)
C:\Users\eozoszy\AppData\Local\Temp\ipykernel_30608\3879195863.py:34: RuntimeWarning: invalid value encountered in divide
  recall = TP / (TP + FN)
C:\Users\eozoszy\AppData\Local\Temp\ipykernel_30608\3879195863.py:35: RuntimeWarning: invalid value encountered in divide
  f1 = 2 * precision * recall / (precision + recall)


In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet().to(device)
model.load_state_dict(torch.load('model_dicom_left_3.pth', map_location=device))

train_accuracy = compute_accuracy(model, train_loader, device)
val_accuracy = compute_accuracy(model, val_loader, device)

print(f'Train Accuracy: {train_accuracy[0] * 100 :.2f}%')
print(f'Train Accuracy Clear: {train_accuracy[1] * 100 :.2f}%')
print()
print(f'Validation Accuracy: {val_accuracy[0] * 100 :.2f}%')
print(f'Validation Accuracy Clear: {val_accuracy[1] * 100 :.2f}%')

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_30608\491998593.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_dicom_left_3.pth', 

Train Accuracy: 95.57%
Train Accuracy Clear: 38.07%

Validation Accuracy: 94.76%
Validation Accuracy Clear: 29.91%


In [12]:
model.load_state_dict(torch.load('model_dicom_left_3.pth', map_location=device))
precision, recall, f1, precision_weighted, recall_weighted, f1_weighted = compute_precision_recall_f1(model, val_loader, device)

if np.isnan(precision).any():
    precision = np.nan_to_num(precision)
if np.isnan(recall).any():
    recall = np.nan_to_num(recall)
if np.isnan(f1).any():
    f1 = np.nan_to_num(f1)

for i in range(28):
    print(f'Class {i}: Precision: {precision[i] * 100:.2f}%, Recall: {recall[i] * 100:.2f}%, F1: {f1[i] * 100:.2f}%')

print(f'Average Precision: {np.mean(precision) * 100:.3f}%')
print(f'Average Recall: {np.mean(recall) * 100:.3f}%')
print(f'Average F1: {np.mean(f1) * 100:.3f}%')
print()
print(f'Weighted Precision: {precision_weighted * 100:.3f}%')
print(f'Weighted Recall: {recall_weighted * 100:.3f}%')
print(f'Weighted F1: {f1_weighted * 100:.3f}%')

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_30608\2962087701.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_dicom_left_3.pth',

Class 0: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 1: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 2: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 3: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 4: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 5: Precision: 64.99%, Recall: 50.66%, F1: 56.94%
Class 6: Precision: 52.15%, Recall: 50.47%, F1: 51.30%
Class 7: Precision: 50.55%, Recall: 50.80%, F1: 50.68%
Class 8: Precision: 52.97%, Recall: 40.94%, F1: 46.18%
Class 9: Precision: 32.33%, Recall: 11.63%, F1: 17.11%
Class 10: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 11: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 12: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 13: Precision: 53.01%, Recall: 47.29%, F1: 49.99%
Class 14: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 15: Precision: 26.65%, Recall: 38.26%, F1: 31.42%
Class 16: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 17: Precision: 51.82%, Recall: 45.88%, F1: 48.66%
Class 18: Precision: 12.61%, Rec

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_30608\3879195863.py:33: RuntimeWarning: invalid value encountered in divide
  precision = TP / (TP + FP)
C:\Users\eozoszy\AppData\Local\Temp\ipykernel_30608\3879195863.py:34: RuntimeWarning: invalid value encountered in divide
  recall = TP / (TP + FN)


In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet().to(device)
model.load_state_dict(torch.load('model_dicom_left_4.pth', map_location=device))

train_accuracy = compute_accuracy(model, train_loader, device)
val_accuracy = compute_accuracy(model, val_loader, device)

print(f'Train Accuracy: {train_accuracy[0] * 100 :.2f}%')
print(f'Train Accuracy Clear: {train_accuracy[1] * 100 :.2f}%')
print()
print(f'Validation Accuracy: {val_accuracy[0] * 100 :.2f}%')
print(f'Validation Accuracy Clear: {val_accuracy[1] * 100 :.2f}%')

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_30608\2976409716.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_dicom_left_4.pth',

Train Accuracy: 94.47%
Train Accuracy Clear: 19.66%

Validation Accuracy: 94.18%
Validation Accuracy Clear: 18.02%


In [14]:
model.load_state_dict(torch.load('model_dicom_left_4.pth', map_location=device))
precision, recall, f1, precision_weighted, recall_weighted, f1_weighted = compute_precision_recall_f1(model, val_loader, device)

if np.isnan(precision).any():
    precision = np.nan_to_num(precision)
if np.isnan(recall).any():
    recall = np.nan_to_num(recall)
if np.isnan(f1).any():
    f1 = np.nan_to_num(f1)

for i in range(28):
    print(f'Class {i}: Precision: {precision[i] * 100:.2f}%, Recall: {recall[i] * 100:.2f}%, F1: {f1[i] * 100:.2f}%')

print(f'Average Precision: {np.mean(precision) * 100:.3f}%')
print(f'Average Recall: {np.mean(recall) * 100:.3f}%')
print(f'Average F1: {np.mean(f1) * 100:.3f}%')
print()
print(f'Weighted Precision: {precision_weighted * 100:.3f}%')
print(f'Weighted Recall: {recall_weighted * 100:.3f}%')
print(f'Weighted F1: {f1_weighted * 100:.3f}%')

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_30608\1094219979.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_dicom_left_4.pth',

Class 0: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 1: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 2: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 3: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 4: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 5: Precision: 58.69%, Recall: 47.55%, F1: 52.53%
Class 6: Precision: 33.19%, Recall: 40.17%, F1: 36.35%
Class 7: Precision: 33.71%, Recall: 38.82%, F1: 36.08%
Class 8: Precision: 42.51%, Recall: 23.24%, F1: 30.05%
Class 9: Precision: 24.75%, Recall: 10.71%, F1: 14.96%
Class 10: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 11: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 12: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 13: Precision: 27.82%, Recall: 1.66%, F1: 3.13%
Class 14: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 15: Precision: 24.19%, Recall: 11.02%, F1: 15.14%
Class 16: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 17: Precision: 43.08%, Recall: 22.70%, F1: 29.73%
Class 18: Precision: 0.00%, Recall

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_30608\3879195863.py:33: RuntimeWarning: invalid value encountered in divide
  precision = TP / (TP + FP)
C:\Users\eozoszy\AppData\Local\Temp\ipykernel_30608\3879195863.py:34: RuntimeWarning: invalid value encountered in divide
  recall = TP / (TP + FN)


In [15]:
# dropout_rate=0.3
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet().to(device)
model.load_state_dict(torch.load('model_dicom_left_5.pth', map_location=device))

train_accuracy = compute_accuracy(model, train_loader, device)
val_accuracy = compute_accuracy(model, val_loader, device)

print(f'Train Accuracy: {train_accuracy[0] * 100 :.2f}%')
print(f'Train Accuracy Clear: {train_accuracy[1] * 100 :.2f}%')
print()
print(f'Validation Accuracy: {val_accuracy[0] * 100 :.2f}%')
print(f'Validation Accuracy Clear: {val_accuracy[1] * 100 :.2f}%')

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_30608\3224584033.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_dicom_left_5.pth',

Train Accuracy: 96.02%
Train Accuracy Clear: 45.46%

Validation Accuracy: 94.97%
Validation Accuracy Clear: 33.60%


In [16]:
# dropout_rate=0.3
model.load_state_dict(torch.load('model_dicom_left_5.pth', map_location=device))
precision, recall, f1, precision_weighted, recall_weighted, f1_weighted = compute_precision_recall_f1(model, val_loader, device)

if np.isnan(precision).any():
    precision = np.nan_to_num(precision)
if np.isnan(recall).any():
    recall = np.nan_to_num(recall)
if np.isnan(f1).any():
    f1 = np.nan_to_num(f1)

for i in range(28):
    print(f'Class {i}: Precision: {precision[i] * 100:.2f}%, Recall: {recall[i] * 100:.2f}%, F1: {f1[i] * 100:.2f}%')

print(f'Average Precision: {np.mean(precision) * 100:.3f}%')
print(f'Average Recall: {np.mean(recall) * 100:.3f}%')
print(f'Average F1: {np.mean(f1) * 100:.3f}%')
print()
print(f'Weighted Precision: {precision_weighted * 100:.3f}%')
print(f'Weighted Recall: {recall_weighted * 100:.3f}%')
print(f'Weighted F1: {f1_weighted * 100:.3f}%')

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_30608\2641582693.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_dicom_left_5.pth',

Class 0: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 1: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 2: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 3: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 4: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 5: Precision: 67.60%, Recall: 54.45%, F1: 60.32%
Class 6: Precision: 60.57%, Recall: 51.94%, F1: 55.93%
Class 7: Precision: 56.36%, Recall: 54.36%, F1: 55.34%
Class 8: Precision: 55.66%, Recall: 43.53%, F1: 48.85%
Class 9: Precision: 31.21%, Recall: 29.65%, F1: 30.41%
Class 10: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 11: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 12: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 13: Precision: 53.08%, Recall: 48.89%, F1: 50.90%
Class 14: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 15: Precision: 31.33%, Recall: 43.85%, F1: 36.55%
Class 16: Precision: 13.75%, Recall: 2.43%, F1: 4.13%
Class 17: Precision: 52.66%, Recall: 47.65%, F1: 50.03%
Class 18: Precision: 22.65%, Re

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_30608\3879195863.py:33: RuntimeWarning: invalid value encountered in divide
  precision = TP / (TP + FP)
C:\Users\eozoszy\AppData\Local\Temp\ipykernel_30608\3879195863.py:34: RuntimeWarning: invalid value encountered in divide
  recall = TP / (TP + FN)


In [17]:
# dropout_rate=0.2
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet().to(device)
model.load_state_dict(torch.load('model_dicom_left_6.pth', map_location=device))

train_accuracy = compute_accuracy(model, train_loader, device)
val_accuracy = compute_accuracy(model, val_loader, device)

print(f'Train Accuracy: {train_accuracy[0] * 100 :.2f}%')
print(f'Train Accuracy Clear: {train_accuracy[1] * 100 :.2f}%')
print()
print(f'Validation Accuracy: {val_accuracy[0] * 100 :.2f}%')
print(f'Validation Accuracy Clear: {val_accuracy[1] * 100 :.2f}%')

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_30608\3328019049.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_dicom_left_6.pth',

Train Accuracy: 96.63%
Train Accuracy Clear: 55.03%

Validation Accuracy: 94.83%
Validation Accuracy Clear: 33.82%


In [18]:
# dropout_rate=0.2
model.load_state_dict(torch.load('model_dicom_left_6.pth', map_location=device))
precision, recall, f1, precision_weighted, recall_weighted, f1_weighted = compute_precision_recall_f1(model, val_loader, device)

if np.isnan(precision).any():
    precision = np.nan_to_num(precision)
if np.isnan(recall).any():
    recall = np.nan_to_num(recall)
if np.isnan(f1).any():
    f1 = np.nan_to_num(f1)

for i in range(28):
    print(f'Class {i}: Precision: {precision[i] * 100:.2f}%, Recall: {recall[i] * 100:.2f}%, F1: {f1[i] * 100:.2f}%')

print(f'Average Precision: {np.mean(precision) * 100:.3f}%')
print(f'Average Recall: {np.mean(recall) * 100:.3f}%')
print(f'Average F1: {np.mean(f1) * 100:.3f}%')
print()
print(f'Weighted Precision: {precision_weighted * 100:.3f}%')
print(f'Weighted Recall: {recall_weighted * 100:.3f}%')
print(f'Weighted F1: {f1_weighted * 100:.3f}%')

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_30608\2511511418.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_dicom_left_6.pth',

Class 0: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 1: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 2: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 3: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 4: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 5: Precision: 62.00%, Recall: 54.91%, F1: 58.24%
Class 6: Precision: 61.16%, Recall: 53.44%, F1: 57.04%
Class 7: Precision: 57.93%, Recall: 42.21%, F1: 48.84%
Class 8: Precision: 47.97%, Recall: 44.77%, F1: 46.32%
Class 9: Precision: 30.57%, Recall: 29.99%, F1: 30.28%
Class 10: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 11: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 12: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 13: Precision: 52.07%, Recall: 54.30%, F1: 53.16%
Class 14: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 15: Precision: 34.61%, Recall: 45.59%, F1: 39.35%
Class 16: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 17: Precision: 54.60%, Recall: 46.20%, F1: 50.05%
Class 18: Precision: 20.31%, Rec

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_30608\3879195863.py:33: RuntimeWarning: invalid value encountered in divide
  precision = TP / (TP + FP)
C:\Users\eozoszy\AppData\Local\Temp\ipykernel_30608\3879195863.py:34: RuntimeWarning: invalid value encountered in divide
  recall = TP / (TP + FN)
C:\Users\eozoszy\AppData\Local\Temp\ipykernel_30608\3879195863.py:35: RuntimeWarning: invalid value encountered in divide
  f1 = 2 * precision * recall / (precision + recall)
